<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/20240518_0932_DNA_Study_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl

In [37]:
# 20240518-0932_DNA_Study_Library

import csv
import glob
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import pandas as pd
from datetime import datetime

# Hard-coded interval scheme
interval_scheme = [
    (1000, 1), (1025, 2), (1050, 3), (1075, 4), (1100, 5),
    (1125, 6), (1150, 7), (1175, 8), (1200, 9), (1225, 10),
    (1250, 11), (1275, 12), (1300, 13), (1325, 14), (1350, 15),
    (1375, 16), (1400, 17), (1425, 18), (1450, 19), (1475, 20),
    (1500, 21), (1525, 22), (1550, 23), (1575, 24), (1600, 25),
    (1625, 26), (1650, 27), (1675, 28), (1700, 29), (1725, 30),
    (1750, 31), (1775, 32), (1800, 33), (1825, 34), (1850, 35),
    (1875, 36), (1900, 37), (1925, 38), (1950, 39), (1975, 40),
    (2000, 41), (2025, 42), (2050, 43), (2075, 44)
]

# Function to assign interval based on birthdate
def assign_interval(birth_date, interval_scheme):
    if pd.isnull(birth_date):
        return 99
    try:
        birth_year = int(birth_date[-4:])
    except ValueError:
        return 99
    for year, interval in interval_scheme:
        if birth_year <= year:
            return interval
    return 99

# Update the GedcomDataset class
class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None  # Initialize anchor_gen1 here

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0]
        last_name = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1  # Declare that we're using the global variable
        anchor_gen1 = self.anchor_gen1  # Update the global variable
        return self.gen_person.strip('@')

    def get_anchor_gen1(self):
        return self.anchor_gen1

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value)
            return cm_value
        except ValueError:
            return 'error'

    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            sort_value = npfx_value.split('&')[1].strip()
            return sort_value
        else:
            return ''

    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC', '').strip('@')

    def get_birth_date(self):
        return self.extractable_detail.get('BIRTH_DATE', '')

    def get_fams(self):
        return self.extractable_detail.get('FAMS', '').strip('@')

# Function definitions
def extract_id(record):
    id_start = record.find('@') + 1
    id_end = record.find('@', id_start)
    return record[id_start:id_end]

def extract_name(record):
    name_start = record.find('1 NAME ') + 6
    name_end = record.find('\n', name_start)
    name = record[name_start:name_end]
    first_name, last_name = name.split('/', 1)
    first_name = first_name[:20]
    last_name = last_name[:20].rstrip('/')
    return last_name.replace(" ", "") + first_name.replace(" ", "")

name_to_id = {}   # Global dictionary to hold name to ID mapping

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    @staticmethod
    def get_standard_name(file_path):
        file_name = file_path.split('/')[-1]
        if '.' in file_name:
            file_name = file_name.rsplit('.', 1)[0]
        standard_name = file_name.replace(' ', '_').lower()
        return standard_name

    def parse_gedcom(self):
        global name_to_id  # Declare name_to_id as global to modify it
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            gedcom_lines = f.readlines()
        current_dataset = None
        npfx_count = 0
        total_count = 0

        for line in gedcom_lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)

                # Populate name_to_id
                individual_name = current_dataset.get_anchor_gen1()
                individual_id = current_dataset.get_gen_person()
                name_to_id[individual_name] = individual_id

            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC', 'FAMS', 'BIRT', 'SEX']:
                    current_key = tag
                    current_dataset.add_extractable_detail(current_key, value)

                elif level == 2 and tag == 'DATE' and current_key == 'BIRT':
                    current_dataset.add_extractable_detail('BIRTH_DATE', value)

                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)

        print(f'GEDCOM contained {total_count} total records')
        print(f'Records tagged and filtered by NPFX: {npfx_count}')

        # First level of filtering: Filter those with NPFX
        for dataset in self.gedcom_datasets:
            if dataset.get_extractable_NPFX():
                self.filter_pool.append(dataset)

        # Check if manual filtering should be applied
        manual_filter_activated = True  # or False depending on your situation

        # Second level of filtering: Apply manual filter from Excel sheet
        if manual_filter_activated:
            import pandas as pd  # Assuming you haven't imported it yet
            try:
                df = pd.read_excel('filtered_ids.xlsx')
            except FileNotFoundError:
                print("filtered_ids.xlsx not found. Skipping second-level manual filter.")
            else:
                manual_filtered_ids = set(df['ID'])
                print(f"Manual filter IDs loaded: {len(manual_filtered_ids) - 1}")

                self.filter_pool = [dataset for dataset in self.filter_pool if dataset.get_gen_person() in manual_filtered_ids]
                print(f"After manual filter, total records: {len(self.filter_pool)}")

    def apply_manual_filter(self):
        manual_filter_activated = True
        if manual_filter_activated:
            import pandas as pd
            try:
                df = pd.read_excel('filtered_ids.xlsx')
                manual_filtered_ids = set(df['ID'].astype(str))  # Ensure IDs are strings
                print(f"Manual filter IDs loaded: {len(manual_filtered_ids)}")
            except FileNotFoundError:
                print("filtered_ids.xlsx not found. Skipping second-level manual filter.")
            else:
                # Debug output to verify IDs before filtering
                print("IDs before manual filter:", [ds.get_gen_person() for ds in self.filter_pool][:10])
                self.filter_pool = [ds for ds in self.filter_pool if str(ds.get_gen_person()) in manual_filtered_ids]
                print("IDs after manual filter:", [ds.get_gen_person() for ds in self.filter_pool][:10])
                print(f"After manual filter, total records: {len(self.filter_pool)}")

    def check_and_apply_exclusion_filter(self):
        """Apply exclusion filter if '/exclude_ids.xlsx' is present."""
        file_path = '/content/exclude_ids.xlsx'  # Updated path
        if os.path.exists(file_path):
            try:
                df_exclude = pd.read_excel(file_path)
                if 'ID' in df_exclude.columns:
                    exclude_ids = set(df_exclude['ID'].astype(str))  # Ensure conversion to string
                    print(f"Exclusion filter IDs loaded: {len(exclude_ids)}")
                    print(f"Sample of IDs to exclude: {list(exclude_ids)[:5]}")  # Print some sample IDs
                    # Apply the exclusion filter
                    initial_count = len(self.filter_pool)
                    self.filter_pool = [ds for ds in self.filter_pool if str(ds.get_gen_person()) not in exclude_ids]
                    print(f"Excluded {initial_count - len(self.filter_pool)} records based on exclusion IDs.")
                else:
                    print("Column 'ID' not found in the Excel file.")
            except Exception as e:
                print(f"Failed to apply exclusion filter: {str(e)}")
        else:
            print(f"No exclusion filter applied, '{file_path}' not found. Check the path and ensure the file is uploaded to Colab.")

def input_prime_surname(last_prime_surname=None):
    if last_prime_surname:
        last_name = input(f"Enter prime_surname (default: {last_prime_surname}): ")
        if not last_name:
            last_name = last_prime_surname
    else:
        last_name = input("Enter prime_surname: ")
    return last_name

def select_gedcom_file():
    gedcom_files = glob.glob('*.ged')
    if not gedcom_files:
        print("No GEDCOM files found.")
        return None

    print("Automatically selecting the first GEDCOM file.")
    return gedcom_files[0]

    while True:
        try:
            selected_num = int(input("Enter the number of the GEDCOM file you want to use: "))
            if 1 <= selected_num <= len(gedcom_files):
                return gedcom_files[selected_num - 1]
            else:
                print("Invalid number. Please enter a valid number from the list.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

gedcom_file_path = select_gedcom_file() # Call the function to let the user select the GEDCOM file
if gedcom_file_path:
    # Use the selected GEDCOM file path to create an instance of the Gedcom class
    gedcom_instance = Gedcom(gedcom_file_path)
    gedcom_instance.parse_gedcom()

    individuals = []  # Initialize the list of individuals

    for dataset in gedcom_instance.filter_pool:    # Iterate over the filter_pool list, add each last name and ID to list
        individual_id = dataset.get_gen_person()
        last_name = dataset.get_anchor_gen1()
        individuals.append((last_name, individual_id))

    print(f'Records tagged and filtered by NPFX: {len(individuals)}')

    with open(gedcom_file_path, 'r') as file:    # Read the GEDCOM file and split it into individual and family records
        data = file.read()
    data = data.split('\n0 ')
    records = {extract_id(record): record for record in data}

def has_both_parents(records, mother_id, father_id):
    return mother_id in records and father_id in records

visited_pairs = set()
generation_table = []

def find_parents(individual_id, generation, records):
    if individual_id not in records:
        return
    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]
    if famc_id not in records:
        return

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if mother_id and mother_id in records and father_id and father_id in records:
        parent_pair = (father_id, mother_id)
        if parent_pair not in visited_pairs:
            visited_pairs.add(parent_pair)
            generation_table.append((generation, parent_pair))

    if mother_id:
        find_parents(mother_id, generation + 1, records)

    if father_id:
        find_parents(father_id, generation + 1, records)

def extract_name(record):
    name_start = record.find('1 NAME ') + 6
    name_end = record.find('\n', name_start)
    name = record[name_start:name_end]
    first_name, last_name = name.split('/', 1)
    first_name = first_name[:20]
    last_name = last_name[:20].rstrip('/')
    return last_name.replace(" ", "") + first_name.replace(" ", "")

def find_distant_ancestors(individual_id, records, path=None):
    path = path if path is not None else []
    if path is None:
        path = [individual_id]
    else:
        path.append(individual_id)

    if individual_id not in records:
        return []

    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]

    if famc_id not in records:
        return [path]

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if father_id is None and mother_id is None:
        return [path]

    paths = []
    if father_id:
        new_path = list(path)
        paths.extend(find_distant_ancestors(father_id, records, new_path))

    if mother_id:
        new_path = list(path)
        paths.extend(find_distant_ancestors(mother_id, records, new_path))

#    print(f"Distant ancestors paths for {individual_id}: {paths}")

    return paths
filtered_datasets = gedcom_instance.filter_pool

#global generation_table
#global visited_pairs

def calculate_score(distant_ancestors_paths, records):
    name_paths = []
    for path in distant_ancestors_paths:
        name_path = [extract_name(records.get(id, '')) for id in path]
        name_paths.append(name_path)

    path_scores = {}
    for idx, name_path in enumerate(name_paths):
        score = 0
        for generation, name in enumerate(name_path):
            if 'Yates' in name:
                score += 1 * (generation + 1)
        path_scores[idx] = score

    if path_scores:
        winning_path_index = max(path_scores, key=path_scores.get)
        winning_path_score = path_scores[winning_path_index]
        winning_path_names = name_paths[winning_path_index]
        winning_path_ids = distant_ancestors_paths[winning_path_index]
    else:
        winning_path_index = None
        winning_path_score = 0
        winning_path_names = []
        winning_path_ids = []

    return winning_path_score, winning_path_names, winning_path_ids

def filter_ancestral_line(winning_path_ids, generation_table):
    matching_table = []

    for generation, pair in generation_table:
        id1, id2 = pair
        if id1 in winning_path_ids or id2 in winning_path_ids:
            matching_table.append((generation, pair))

    return matching_table

def filter_ancestral_line(winning_path_ids, generation_table):
    matching_table = []
    for generation, pair in generation_table:
        id1, id2 = pair
        if id1 in winning_path_ids or id2 in winning_path_ids:
            matching_table.append((generation, pair))
    return matching_table

# Main Loop
for dataset in filtered_datasets:
    individual_id = dataset.get_gen_person()

    visited_pairs = set()
    generation_table = []

    find_parents(individual_id, 1, records)
    distant_ancestors_paths = find_distant_ancestors(individual_id, records)
    winning_path_score, winning_path_names, winning_path_ids = calculate_score(distant_ancestors_paths, records)
    filtered_ancestral_line = filter_ancestral_line(winning_path_ids, generation_table)
    filtered_ancestral_line.sort(key=lambda x: x[0])
    filtered_ancestral_line_names = []
    for generation, pair in filtered_ancestral_line:
        name_pair = [extract_name(records.get(id, '')) for id in pair]
        formatted_name_pair = f"{name_pair[0]}&{name_pair[1]}"
        filtered_ancestral_line_names.append(formatted_name_pair)

    filtered_ancestral_line_names.reverse()
#    filtered_ancestral_line_str = "|".join(filtered_ancestral_line_names)
#    print(f"Filtered Ancestral Line for {individual_id}: {filtered_ancestral_line_str}")

def process_individual(individual_id, gedcom_instance, records, interval_scheme):
    global generation_table
    global visited_pairs
    global anchor_gen1  # Declare that we're using the global variable

    generation_table = []
    visited_pairs = set()

    find_parents(individual_id, 1, records)
    distant_ancestors_paths = find_distant_ancestors(individual_id, records)
    winning_path_score, winning_path_names, winning_path_ids = calculate_score(distant_ancestors_paths, records)
    filtered_ancestral_line = filter_ancestral_line(winning_path_ids, generation_table)
    filtered_ancestral_line.sort(key=lambda x: x[0])
    filtered_ancestral_line_names = []
    birth_date = None  # Initialize birth_date

    # Extract cm_value, sort_value, anchor_gen1, and birth_date
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            cm_value = dataset.get_extractable_cm()
            sort_value = dataset.get_extractable_sort()
            anchor_gen1 = dataset.get_anchor_gen1()  # Update anchor_gen1 locally here
            birth_date = dataset.extractable_detail.get('BIRTH_DATE')  # Get birth date
            break
    else:
        cm_value = 'N/A'
        sort_value = 'N/A'
        birth_date = 'N/A'  # Set default if not found

    if anchor_gen1 is not None:
        filtered_ancestral_line_names.insert(0, anchor_gen1)

    ancestors_data = []

    # Extract ancestor information
    for generation, pair in filtered_ancestral_line:
        name_pair = [extract_name(records.get(id, '')) for id in pair]
        formatted_name_pair = f"{name_pair[0]}&{name_pair[1]}"
        filtered_ancestral_line_names.append(formatted_name_pair)

        # Extract details for each ancestor in the pair
        for ancestor_id in pair:
            if ancestor_id in records:
                ancestor_record = records[ancestor_id]
                ancestor_name = extract_name(ancestor_record)
                ancestor_sex = 'M' if '1 SEX M' in ancestor_record else 'F' if '1 SEX F' in ancestor_record else ''
                birth_date_start = ancestor_record.find('2 DATE ') + 7
                birth_date_end = ancestor_record.find('\n', birth_date_start)
                ancestor_birth_date = ancestor_record[birth_date_start:birth_date_end].strip() if birth_date_start != -1 else 'N/A'
                ancestor_date_interval = assign_interval(ancestor_birth_date, interval_scheme)

                ancestors_data.append({
                    'ID': ancestor_id,
                    'Name': ancestor_name,
                    'Sex': ancestor_sex,
                    'Birth Date': ancestor_birth_date,
                    'Date Interval': ancestor_date_interval
                })

    filtered_ancestral_line_names.reverse()
    filtered_ancestral_line_str = "~~~".join(filtered_ancestral_line_names)

    # Assign date interval
    date_interval = assign_interval(birth_date, interval_scheme)

    individual_data = {
        'ID': individual_id,
        'Name': extract_name(records[individual_id]),
        'Sex': dataset.extractable_detail.get('SEX', ''),
        'Birth Date': birth_date,
        'Date Interval': date_interval,
        'FAMS': dataset.get_fams(),
        'FAMC': dataset.get_extractable_FAMC(),
        'Ancestors': ancestors_data  # Add ancestors data to individual data
    }

    return individual_data, filtered_ancestral_line_str

# Initialize the dictionary to store individual data
DNA_Study_Library = {}

# Process individuals and populate the dictionary
for dataset in gedcom_instance.filter_pool:  # Assuming filter_pool is iterable
    individual_id = dataset.get_gen_person()

    # Reset global variables for each new individual
    visited_pairs.clear()
    generation_table = []

    # Process Individual and Get Data
    individual_data, filtered_ancestral_line_str = process_individual(individual_id, gedcom_instance, records, interval_scheme)

    # Store the data in the dictionary
    DNA_Study_Library[individual_id] = individual_data

# Output the first 5 records in a columnar format
print(f"{'ID':<10}{'Name':<20}{'Sex':<5}{'Birth Date':<12}{'Date Interval':<15}{'FAMS':<10}{'FAMC':<10}{'Ancestors':<30}")
for idx, (individual_id, data) in enumerate(DNA_Study_Library.items()):
    if idx >= 25:
        break
    # Replace None values with empty strings
    name = data['Name'] or ''
    sex = data['Sex'] or ''
    birth_date = data['Birth Date'] or ''
    date_interval = data['Date Interval'] or ''
    fams = data['FAMS'] or ''
    famc = data['FAMC'] or ''
    ancestors = ', '.join([f"{ancestor['Name']} ({ancestor['ID']})" for ancestor in data['Ancestors']])
    print(f"{individual_id:<10}{name:<20}{sex:<5}{birth_date:<12}{date_interval:<15}{fams:<10}{famc:<10}{ancestors:<30}")





Automatically selecting the first GEDCOM file.
GEDCOM contained 55730 total records
Records tagged and filtered by NPFX: 1045
filtered_ids.xlsx not found. Skipping second-level manual filter.
Records tagged and filtered by NPFX: 1045
ID        Name                Sex  Birth Date  Date Interval  FAMS      FAMC      Ancestors                     
I73       YatesJamesRobert    M    24 MAR 2018 42             F90       F6        YatesHarryElmer (I4), TingleyOrliaGeorgeann (I27), YatesJamesWilson (I5), OttMaryEllen (I11), YatesJohnE (I6), RobersonElizabeth (I10), YatesJohn (I3097), SwiftMary (I3098), YatesJohn (I1498), SearchingStill (I22324), YatesBenjamin (I19677), SearchingStill (I54742), YatesBenjamin (I18175), SearchingStill (I55721)
I80       YatesPatriciaLynn   F    1 JUL 1951  40             F98       F3        YatesCarlBernell (I3), CarpenterLuellaMay (I7), YatesHarryElmer (I4), LaswellJessieLeah (I12), YatesJamesWilson (I5), OttMaryEllen (I11), YatesJohnE (I6), RobersonElizabeth (

In [15]:
#Super visualize 20240517-2008

import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Alignment, PatternFill

def process_data(filepath):
    """Load data, process it to expand 'Yates DNA Ancestral Line', calculate FQ and QI, sort by frequency, and save results with visual separators."""
    # Load the DataFrame
    df = pd.read_excel(filepath)

    # Prepare to expand and process
    expanded_data = []
    parents_stack = []
    starting_plane = 0

    # Process each row to expand and calculate
    for index, row in df.iterrows():
        nodes = row['Yates DNA Ancestral Line'].split('~~~')
        for i, node in enumerate(nodes):
            plane = i + starting_plane
            if len(parents_stack) > i:
                parents_stack[i] = node
            else:
                parents_stack.append(node)
            parent = parents_stack[i-1] if i > 0 else None
            expanded_data.append({
                'Gen #': plane,
                'Offspring & Spouse': node,
                'Parents': parent,
                'cM': row['cM'],
                'ID': row.get('ID', '')  # Ensure ID is optionally included
            })

    # Convert the list to a DataFrame
    expanded_df = pd.DataFrame(expanded_data)

    # Group by 'Gen #' and 'Parents' and calculate FQ and QI
    results_df = expanded_df.groupby(['Gen #', 'Parents', 'Offspring & Spouse']).agg(
        FQ=('Offspring & Spouse', 'size'),
        QI=('cM', 'mean')
    ).reset_index()
    results_df['QI'] = pd.to_numeric(results_df['QI'], errors='coerce').fillna(0).astype(int)

    # Filter results where FQ is greater than or equal to 3
    final_results_df = results_df[results_df['FQ'] >= 3].copy()

    # Sort data by 'Gen #', 'FQ', and then 'Offspring & Spouse' for proper ranking
    final_results_df.sort_values(by=['Gen #', 'FQ', 'Offspring & Spouse'], ascending=[True, False, True], inplace=True)

    # Create an Excel file with openpyxl
    wb = Workbook()
    ws = wb.active

    # Adding rows to worksheet
    headers = ['Gen #', 'Parents', 'Offspring & Spouse', 'FQ', 'QI']
    ws.append(headers)

    # Initialize variables to track last seen parent and generation
    last_gen = None
    last_parent = None
    parent_seen = set()

    for r in dataframe_to_rows(final_results_df, index=False, header=False):
        current_gen, current_parent = r[0], r[1]

        if current_parent not in parent_seen:
            parent_seen.add(current_parent)
            ws.append(r)
        else:
            r[1] = ''  # Clear parent name for repeated entries within the same generation
            ws.append(r)

    # Adding borders, alignment, and fill colors for readability
    thin = Side(border_style="thin", color="000000")
    thick = Side(border_style="thick", color="000000")
    center_alignment = Alignment(horizontal="center", vertical="center")

    # Define fill colors for different generations
    colors = ['FFCCCB', 'CCFFCC']  # Alternate colors for different generations
    color_idx = 0

    # Apply borders, alignment, and fill colors within the sheet
    last_gen = None
    for row in ws.iter_rows(min_row=2, max_col=len(headers), max_row=ws.max_row):
        current_gen = row[0].value
        current_parent = row[1].value

        if last_gen is not None and current_gen != last_gen:
            color_idx = (color_idx + 1) % len(colors)
            for cell in row:
                cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
        else:
            for cell in row:
                if cell.column_letter != 'B':  # Avoid placing borders on the parent column
                    cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        fill = PatternFill(start_color=colors[color_idx], end_color=colors[color_idx], fill_type="solid")
        for cell in row:
            cell.alignment = center_alignment
            cell.fill = fill

        last_gen = current_gen

    # Apply thick border after each parent group
    for row in ws.iter_rows(min_row=2, max_col=len(headers), max_row=ws.max_row):
        current_parent = row[1].value
        if current_parent and current_parent != last_parent:
            for cell in row:
                cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
        last_parent = current_parent

    # Save the workbook
    output_file = '/content/4_generations_chart_styled.xlsx'
    wb.save(output_file)
    print(f"Results successfully saved with styled borders to {output_file}")

# Specify the file path for the data
input_file_path = '/content/3_generations_fq-qi.xlsx'

# Process the data
process_data(input_file_path)




Results successfully saved with styled borders to /content/4_generations_chart_styled.xlsx
